In [159]:
import pandas as pd
import numpy as np
import wquantiles
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [161]:
# Summon Households in the Berkeley-Albany PUMA zone (the smallest geography for Berkeley)
ca_households = pd.read_csv("ACS_2023/csv_hca/psam_h06.csv")
# PUMAs_of_interest = [111,112,113,123] # Oakland codes
# PUMAs_of_interest =  [501,502,503,504,505,506,507] #San Francisco
PUMAs_of_interest = [101] # Berkeley, Albany code : 06 - 00101
households_set = ca_households[ca_households['PUMA'].isin(PUMAs_of_interest)]

20    NaN
35    NaN
105   NaN
197   NaN
237   NaN
Name: HHLDRRAC1P, dtype: float64

In [130]:
# SERIALNO : Unique ID for Census ACS respondants, in this case, households.
households_set.head()

,RT,SERIALNO,DIVISION,PUMA,REGION,STATE,ADJHSG,ADJINC,WGTP,NP,TYPEHUGQ,ACCESSINET,ACR,AGS,BATH,BDSP,BLD,BROADBND,COMPOTHX,CONP,DIALUP,ELEFP,ELEP,FS,FULFP,FULP,GASFP,GASP,HFL,HISPEED,HOTWAT,INSP,LAPTOP,MHP,MRGI,MRGP,MRGT,MRGX,OTHSVCEX,REFR,RMSP,RNTM,RNTP,RWAT,RWATPR,SATELLITE,SINK,SMARTPHONE,SMP,STOV,TABLET,TEL,TEN,VACS,VALP,VEH,WATFP,WATP,YRBLT,CPLT,FINCP,FPARC,GRNTP,GRPIP,HHL,HHLANP,HHLDRAGEP,HHLDRHISP,HHLDRRAC1P,HHT,HHT2,HINCP,HUGCL,HUPAC,HUPAOC,HUPARC,KIT,LNGI,MULTG,MV,NOC,NPF,NPP,NR,NRC,OCPIP,PARTNER,PLM,PLMPRP,PSF,R18,R60,R65,RESMODE,SMOCP,SMX,SRNT,SVAL,TAXAMT,WIF,WKEXREL,WORKSTAT,FACCESSP,FACRP,FAGSP,FBATHP,FBDSP,FBLDP,FBROADBNDP,FCOMPOTHXP,FCONP,FDIALUPP,FELEP,FFINCP,FFSP,FFULP,FGASP,FGRNTP,FHFLP,FHINCP,FHISPEEDP,FHOTWATP,FINSP,FKITP,FLAPTOPP,FMHP,FMRGIP,FMRGP,FMRGTP,FMRGXP,FMVP,FOTHSVCEXP,FPLMP,FPLMPRP,FREFRP,FRMSP,FRNTMP,FRNTP,FRWATP,FRWATPRP,FSATELLITEP,FSINKP,FSMARTPHONP,FSMOCP,FSMP,FSMXHP,FSMXSP,FSTOVP,FTABLETP,FTAXP,FTELP,FTENP,FVACSP,FVALP,FVEHP,FWATP,FYRBLTP,WGTP1,WGTP2,WGTP3,WGTP4,WGTP5,WGTP6,WGTP7,WGTP8,WGTP9,WGTP10,WGTP11,WGTP12,WGTP13,WGTP14,WGTP15,WGTP16,WGTP17,WGTP18,WGTP19,WGTP20,WGTP21,WGTP22,WGTP23,WGTP24,WGTP25,WGTP26,WGTP27,WGTP28,WGTP29,WGTP30,WGTP31,WGTP32,WGTP33,WGTP34,WGTP35,WGTP36,WGTP37,WGTP38,WGTP39,WGTP40,WGTP41,WGTP42,WGTP43,WGTP44,WGTP45,WGTP46,WGTP47,WGTP48,WGTP49,WGTP50,WGTP51,WGTP52,WGTP53,WGTP54,WGTP55,WGTP56,WGTP57,WGTP58,WGTP59,WGTP60,WGTP61,WGTP62,WGTP63,WGTP64,WGTP65,WGTP66,WGTP67,WGTP68,WGTP69,WGTP70,WGTP71,WGTP72,WGTP73,WGTP74,WGTP75,WGTP76,WGTP77,WGTP78,WGTP79,WGTP80
11,H,2019GQ0000081,9,111,4,6,1195583,1207712,0,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
19,H,2019GQ0000219,9,112,4,6,1195583,1207712,0,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
81,H,2019GQ0000925,9,113,4,6,1195583,1207712,0,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
125,H,2019GQ0001389,9,112,4,6,1195583,1207712,0,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [131]:
# HINCP = Household income (past 12 months, use ADJINC to adjust HINCP to constant dollars)
# ADJINC = Adjustment factor for income and earnings dollar amounts (6 implied decimal places)
# Divide ADJINC by 1,000,000 to obtain the inflation adjustment factor and multiply it to the PUMS variable value to adjust it to 2023 dollars
households_set['Adjusted_Income'] = (households_set['HINCP'].astype(float) * households_set['ADJINC'].astype(float)) / 1000000

/var/folders/2k/1029_3wx62vgt24s0p06bzm80000gn/T/ipykernel_71853/1358105620.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  households_set['Adjusted_Income'] = (households_set['HINCP'].astype(float) * households_set['ADJINC'].astype(float)) / 1000000


In [132]:
# Making a column called building type (bldtype) where the home of every household / unit is corresponded with the following strings
conditions = [
    households_set['BLD'] == 1,
    households_set['BLD'] == 2,
    households_set['BLD'] == 3,
    households_set['BLD'].isin([4, 5]),
    households_set['BLD'].isin([6, 7]),
    households_set['BLD'] == 8,
    households_set['BLD'] == 9,
    households_set['BLD'] == 10
]

choices_type = [
    "Mobile home or trailer",
    "Single-family detached",
    "Single-family attached",
    "2-4 units",
    "5-19 units",
    "20-49 units",
    "50+ units",
    "Boat/ RV/ Van"
]
households_set["bldtype"] = np.select(conditions, choices_type, default="unknown")

/var/folders/2k/1029_3wx62vgt24s0p06bzm80000gn/T/ipykernel_71853/3671717487.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  households_set["bldtype"] = np.select(conditions, choices_type, default="unknown")


# Explaining the data:
Note that although these are the values below are per row or ACS response, the numbers below are not the proportion of the population. 
Each row and respondent to the Census questionares is assigned a weight which determines how many households this respondent is represenative of. 
This is how the Census estimates housing and population during non-decinneal years by sending questionaires to a sample and assigning them a weight
It is not practical to send every citizen every year a very detailed Census survey, but they can be sampled in ways that can offer estimations of the broader public.
# Example: 
Suppose that the Census surveys and gets responses back from 1000 single-family residents (SFR) and 100 apartment residents (AR). But the Census knows the 50% of the city is single-family and 50% apartments. The weight for SFH owners is 0.5 (50%) / 1,000 = 0.0005 and the weight for Apt. residents is 0.5 / 100 = 0.005. When we calculate the median household incomes, the individual apartment household response will be assigned as a greater represenative value or "weight" for apartment households overall than the individual SFH owner response will be for SFH owners overall. Calculating medians without weights supposes all numbers (or responses in this case) are equal. The median of incomes in this program will be weighted medians to account for the discrepency in responses.

In [134]:
# Number of ACS household respondants living in the array of building types in Berkeley-Albany

households_set["bldtype"].value_counts()

bldtype
Single-family detached    4148
2-4 units                 1360
50+ units                 1147
5-19 units                1081
20-49 units                779
unknown                    735
Single-family attached     497
Mobile home or trailer      39
Boat/ RV/ Van                7
Name: count, dtype: int64

In [135]:
# Same thing with building ages; classify them with corresponding values
conditions = [
    households_set['YRBLT'] <= 1950,
    (households_set['YRBLT'] > 1950) & (households_set['YRBLT'] <= 1970),
    (households_set['YRBLT'] > 1970) & (households_set['YRBLT'] <= 1990),
    (households_set['YRBLT'] > 1990) & (households_set['YRBLT'] <= 2009),
    households_set['YRBLT'] > 2009
]
choices_age = [
    "Pre-1950",
    "1951 - 1970",
    "1971 - 1990",
    "1991 - 2009",
    "2010 - 2023"
]
households_set["bldage"] = np.select(conditions, choices_age, default="unknown")


/var/folders/2k/1029_3wx62vgt24s0p06bzm80000gn/T/ipykernel_71853/197424395.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  households_set["bldage"] = np.select(conditions, choices_age, default="unknown")


In [136]:
# Number of ACS household respondants living in the array of homes by age of building in Berkeley-Albany
households_set["bldage"].value_counts()

bldage
Pre-1950       5304
1951 - 1970    1758
1971 - 1990     960
unknown         735
1991 - 2009     586
2010 - 2023     450
Name: count, dtype: int64

In [137]:
# Results will contain the final product: Median Household Income by Density and Age
results = pd.DataFrame(index=choices_type, columns=choices_age)
# This will show the numbered of (weighted) samples of the above figure
results_nrow = pd.DataFrame(index=choices_type, columns=choices_age)
                            
for unit in choices_type:
    for age in choices_age:
        # Filter out units by age and density level, but only use household incomes that are known
        iter_set = households_set[
            (households_set['bldtype'] == unit) &
            (households_set['bldage'] == age) &
            (~households_set['Adjusted_Income'].isna() & 
            households_set['Adjusted_Income']> 0)
        ]
        # Sum the housing unit weights for this type of household (the weights estimate the represenative proportion of the sample)
        # If weight is under 1, data is missing likely for this cohort for privacy reasons or low sampling. Disregard this response
        if iter_set["WGTP"].sum(skipna=True) < 1:
            results.loc[unit, age] = -1
        # Else, calculate the Household's Income using a represenative weight of the sample response
        else:
            results.loc[unit, age]  = wquantiles.median(iter_set["Adjusted_Income"], iter_set["WGTP"])
        # Sum the weights to see how represenative each housing type/age is
        results_nrow.loc[unit, age] = iter_set["WGTP"].sum(skipna=True)

In [138]:
# The Results (excludes households reporting $0 income)
# Format the float columns as dollars
results_styled = results.style.format({
    'Pre-1950':  lambda x: '${:,.2f}'.format(x) if x != -1 else 'NA', 
    '1951 - 1970': lambda x: '${:,.2f}'.format(x) if x != -1 else 'NA',  
    '1971 - 1990': lambda x: '${:,.2f}'.format(x) if x  != -1 else 'NA',   
    '1991 - 2009':  lambda x: '${:,.2f}'.format(x) if x != -1 else 'NA', 
    '2010 - 2023':  lambda x: '${:,.2f}'.format(x) if x  != -1 else 'NA',   
})
results_styled

,Pre-1950,1951 - 1970,1971 - 1990,1991 - 2009,2010 - 2023
Mobile home or trailer,"$34,927.30","$37,941.14","$104,084.94","$81,365.04","$213,555.22"
Single-family detached,"$136,787.95","$137,803.36","$155,407.47","$134,952.97","$165,746.34"
Single-family attached,"$84,875.52","$121,576.67","$139,793.78","$134,356.68","$194,321.77"
2-4 units,"$78,770.23","$65,121.36","$62,553.99","$42,798.54","$27,684.06"
5-19 units,"$64,008.74","$77,005.31","$69,353.48","$109,723.40","$104,868.10"
20-49 units,"$75,884.43","$76,246.83","$67,451.60","$88,119.03","$113,870.49"
50+ units,"$25,309.54","$54,398.99","$78,193.98","$139,968.33","$112,257.63"
Boat/ RV/ Van,"$78,647.34","$379,857.80","$95,208.63",NA,"$92,301.77"


In [139]:
# Weighted Results Sample Size
# This is the Numerical Representation of how many households the income data is informed by
results_nrow.head(len(results_nrow))

,Pre-1950,1951 - 1970,1971 - 1990,1991 - 2009,2010 - 2023
Mobile home or trailer,167,234,223,132,63
Single-family detached,56376,8568,5117,2376,1573
Single-family attached,4038,1053,1515,905,897
2-4 units,17091,6135,3360,777,371
5-19 units,10313,7110,2879,1240,419
20-49 units,4883,6243,1878,1561,1042
50+ units,3550,5936,3354,3988,4818
Boat/ RV/ Van,45,40,10,0,50


Re-calculating but including households reporting $0 income (i.e. households using their units to house in-laws/kids etc. Just drags the median down. Not the most useful for this argument.)

In [141]:
# Results will contain the final product: Median Household Income by Density and Age
results = pd.DataFrame(index=choices_type, columns=choices_age)
# This will show the numbered of (weighted) samples of the above figure
results_nrow = pd.DataFrame(index=choices_type, columns=choices_age)
                            
for unit in choices_type:
    for age in choices_age:
        # Filter out units by age and density level, but only use household incomes that are known
        iter_set = households_set[
            (households_set['bldtype'] == unit) &
            (households_set['bldage'] == age) &
            (~households_set['Adjusted_Income'].isna() )
        ]
        # Sum the housing unit weights for this type of household (the weights estimate the represenative proportion of the sample)
        # If weight is under 1, data is missing likely for this cohort for privacy reasons or low sampling. Disregard this response
        if iter_set["WGTP"].sum(skipna=True) < 1:
            results.loc[unit, age] = -1
        # Else, calculate the Household's Income using a represenative weight of the sample response
        else:
            results.loc[unit, age]  = wquantiles.median(iter_set["Adjusted_Income"], iter_set["WGTP"])
        # Sum the weights to see how represenative each housing type/age is
        results_nrow.loc[unit, age] = iter_set["WGTP"].sum(skipna=True)

In [142]:
# The Results (includes households reporting $0 income)
# Format the float columns as dollars
results_styled = results.style.format({
    'Pre-1950':  lambda x: '${:,.2f}'.format(x) if x != -1 else 'NA', 
    '1951 - 1970': lambda x: '${:,.2f}'.format(x) if x != -1 else 'NA',  
    '1971 - 1990': lambda x: '${:,.2f}'.format(x) if x  != -1 else 'NA',   
    '1991 - 2009':  lambda x: '${:,.2f}'.format(x) if x != -1 else 'NA', 
    '2010 - 2023':  lambda x: '${:,.2f}'.format(x) if x  != -1 else 'NA',   
})
results_styled

,Pre-1950,1951 - 1970,1971 - 1990,1991 - 2009,2010 - 2023
Mobile home or trailer,"$34,048.45","$37,941.14","$104,084.94","$81,365.04","$213,555.22"
Single-family detached,"$133,780.86","$132,840.33","$151,875.97","$134,952.97","$165,746.34"
Single-family attached,"$84,698.32","$121,576.67","$135,244.78","$134,356.68","$194,321.77"
2-4 units,"$77,339.12","$63,778.76","$61,291.38","$42,798.54","$27,684.06"
5-19 units,"$61,169.92","$76,555.29","$65,118.48","$109,723.40","$97,565.50"
20-49 units,"$75,596.15","$74,907.00","$67,451.60","$80,664.96","$113,870.49"
50+ units,"$24,368.08","$53,975.71","$75,650.10","$139,106.95","$106,999.64"
Boat/ RV/ Van,"$78,647.34","$379,857.80","$95,208.63",NA,"$92,301.77"


In [143]:
# Weighted Results Sample Size
# This is the Numerical Representation of how many households the income data is informed by
results_nrow.head(len(results_nrow))

,Pre-1950,1951 - 1970,1971 - 1990,1991 - 2009,2010 - 2023
Mobile home or trailer,192,234,223,132,63
Single-family detached,57025,8741,5202,2376,1573
Single-family attached,4054,1053,1608,905,897
2-4 units,17385,6304,3410,777,371
5-19 units,10481,7273,3067,1240,426
20-49 units,4964,6413,1878,1613,1042
50+ units,3704,6005,3443,4019,4941
Boat/ RV/ Van,45,40,10,0,50
